In [1]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from __future__ import print_function
import torch

In [3]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('/home/luc/TRAVAIL/DeepGED/Acyclic/trainset_0.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[1]
G2 = Gs[9]
print(y[13],y[23])
#nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')
#plt.figure()

#nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

import extended_label
for g in Gs:
    extended_label.compute_extended_labels(g)
for v in Gs[10].nodes():
        print(Gs[10].nodes[v])

edge max label 1
64.4 52.5
{'atom': 'C', 'label': ['C'], 'attributes': ['0.0000', '0.0000', '0.0000'], 'extended_label': ['C_1C']}
{'atom': 'C', 'label': ['C'], 'attributes': ['0.0000', '0.0000', '0.0000'], 'extended_label': ['C_1C']}
{'atom': 'C', 'label': ['C'], 'attributes': ['0.0000', '0.0000', '0.0000'], 'extended_label': ['C_1O']}
{'atom': 'C', 'label': ['C'], 'attributes': ['0.0000', '0.0000', '0.0000'], 'extended_label': ['C_1C1C1O']}
{'atom': 'O', 'label': ['O'], 'attributes': ['0.0000', '0.0000', '0.0000'], 'extended_label': ['O_1C1C']}


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import franck_wolfe

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,node_label='label'):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.node_label=node_label
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print('nb labels:',self.nb_labels,' ; nb_edge_labels:',self.nb_edge_labels)
        self.device=torch.device("cuda:0")
        nb_node_pair_label=self.nb_labels*(self.nb_labels-1)/2.0
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(int(nb_node_pair_label+1),requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del        
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        #self.coef_dist=nn.Parameter(torch.tensor(1.0,requires_grad=True,device=self.device))
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.empty((len(GraphList),card_max*card_max),dtype=torch.int,device=self.device)
        self.labels=torch.empty((len(GraphList),card_max),dtype=torch.int,device=self.device)
        print(self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,0:l.shape[0]]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        
    def forward(self, input):        
        ged=torch.zeros(len(input)).to(self.device) 
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        #alpha=self.coef_dist*self.coef_dist
       
        
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        for k in range(len(input)):            
            g1=input[k][0]
            g2=input[k][1]
            n=self.card[g1]
            m=self.card[g2]
            C=self.construct_cost_matrix(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)      
            
            #S=self.mapping_from_similarity(C,n,m) # SVD or iterated power according to compute major axis
            S=self.mapping_from_cost(C,n,m) # FW
            v=torch.flatten(S)
            
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                normalize_factor=nodeInsDel*(n+m)+edgeInsDel*(nb_edge1 +nb_edge2)                                    
            c=torch.diag(C)
            D=C-torch.eye(C.shape[0],device=self.device)*c
            ged[k]=(.5*v.T@D@v+c.T@v)/normalize_factor
       
            
        
        return ged
    
    def from_weighs_to_costs(self):
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        ce=self.edge_weighs*self.edge_weighs
        total_cost=(cn.sum()+ce.sum())
        cn=cn/total_cost
        ce=ce/total_cost
        edgeInsDel=ce[-1]

        node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        node_costs=node_costs+node_costs.T
        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return node_costs,cn[-1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v][self.node_label][0] in node_labels:
                    node_labels.append(G.nodes[v][self.node_label][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print(node_labels)
        print(dict,len(dict))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=int,weight='bond_type').todense(),dtype=torch.int)        
        lab=[dict[G.nodes[v][self.node_label][0]] for v in nx.nodes(G)]
   
        return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

    def construct_cost_matrix(self,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
        A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
        A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
        A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)
        
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
#                    C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1),alpha=edge_costs[k][l])
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1)
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

        l1=self.labels[g1][0:n]
        l2=self.labels[g2][0:m]
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
        #import ipdb; ipdb.set_trace()
        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
    
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1)
    
    def mapping_from_cost(self,C,n,m):        
        c=torch.diag(C)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=franck_wolfe.lsape(c.view(n+1,m+1),0.5,10)
        #x0=svd.eps_assigment_from_mapping(torch.exp(-0.5*c.view(n+1,m+1)),10).view((n+1)*(m+1),1) # a améliorer.
        #Cred,delta=franck_wolfe.from_lsape_to_lsap(c.view(n+1,m+1))
        #S=franck_wolfe.from_costs_to_similarities(Cred,-.5)
        #S=torch.exp(-.5*Cred)
        #s=franck_wolfe.sinkhorn(S,10)
        #x0=franck_wolfe.lsape_recover_sol_from_lsap(s,delta).view((n+1)*(m+1),1)
        x=franck_wolfe.franck_wolfe(x0,D,c,2,10,n,m)
        def print_grad(grad):
            if(grad.norm()!= 0.0):
                print(grad)
        
       # x.register_hook(print_grad)
        return x
    
      
 
    
            
#print(model(input))
#print(max([G.order() for G in Gs]),len(Gs))
#print('toto')

In [5]:
from regression import KnnRegressFromGED as regress
from regression import train_set_for_knnregression as create_dataset

nb=len(Gs)
#class1=torch.tensor([k for k in range(len(y)) if y[k]==1])
#class2=torch.tensor([k for k in range(len(y)) if y[k]==0])

train_size=70
nb_test=10
train_graphs=torch.randperm(nb)[0:train_size]
print('train graphs:', train_graphs)



#couples=[]
#for k in range(nb_elt):
#    if (y[data[k][0]]!=y[data[k][1]]):
#        yt[k]=-1.0        
#data,yt=train_set_for_kernel_ridge(train_graphs,y,train_size)
data,yt=create_dataset(train_graphs,y,train_size,nb_test)
print('data=',data,data.shape[0])
print('yt=',yt)
#print(couples[1:2])

torch.cuda.empty_cache()
if torch.cuda.is_available():
    device = torch.device("cuda:0")          # a CUDA device object    


train graphs: tensor([ 44, 106,  57, 135,  55,  62,  56, 139, 117,  30, 113,  59, 156,  25,
         54, 163, 159,  21,  10,  64,   4, 131,  47, 149,  18,  33,   7,  11,
        125, 119, 143,  74,  34, 101, 126,   0,  75,  87, 137, 132,  85, 111,
        104,  27,  38,  97,  42,  70,  84,  35,  81,  98, 100, 160,  16,  67,
          9,   2, 121,  49, 162, 130,  76, 148,  50,  15,  80,  68, 140, 134])
data= tensor([[162,  44],
        [162, 106],
        [162,  57],
        ...,
        [134,   2],
        [134, 121],
        [134,  49]], dtype=torch.int32) 600
yt= tensor([ 91.5000, 122.0000, 144.2000, 162.0000, 120.3000, 139.0000, 145.0000,
        155.5000, 165.0000,  92.0000, 109.5000, 132.0000, 173.0000,  59.5000,
        123.5000, 240.0000, 188.5000,  70.3000,  32.0000, 120.0000,  39.0000,
        157.0000,  86.3000, 250.0000, 156.0000, 112.5000, 135.0000,  63.0000,
        195.0000, 170.0000, 163.0000, 117.1000, 118.5000, 142.0000, 215.0000,
         14.0000, 107.0000, 124.0000, 

In [ ]:










from collections import OrderedDict
from triangular_losses import TriangularConstraint as triangular_constraint
#if torch.cuda.device_count() > 1:
#  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#  model = nn.DataParallel(model)
#model.to(device)
#regress.to(device)
def regression(model,data,yt,nb_iter):

    input=data.to(device)
    target=yt.to(device) 
    
    criterion = torch.nn.MSELoss()
    criterionTri=triangular_constraint()
    
        
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
    print(device)

#torch.cat((same_class[0:20],diff_class[0:20]),0).to(device)
    
#torch.ones(40,device=device)
#target[20:]=-1.0
#target=(yt[0:20]).to(device)
    InsDel=np.empty((nb_iter,2))
    node_costs,nodeInsDel,edge_costs,edgeInsDel=model[0].from_weighs_to_costs()
    nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    #print('alpha=',regress.coef_dist*regress.coef_dist*10.0)
    
    
    for t in range(nb_iter):    
        # Forward pass: Compute predicted y by passing data to the model    
        node_costs,nodeInsDel,edge_costs,edgeInsDel=model[0].from_weighs_to_costs()

        y_pred = model(input).to(device)
            
        # Compute and print loss
        loss = criterion(y_pred, target[-nb_test:]).to(device)                          
        #loss = criterion(y_pred, target).to(device)                          
        triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)
        
        loss=loss*(1+triangularInq)
        loss.to(device)
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        rmse=np.sqrt(loss.item()/nb_test)
        loss_plt[t]=rmse
        print(t, rmse)   
        k=0
        for p in range(node_costs.shape[0]):
            for q in range(p+1,node_costs.shape[0]):
                nodeSub[t][k]=node_costs[p][q]
                k=k+1
        k=0
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        if t % 20 == 0 or t==0:                         
            #print('Distances: ',y_pred)
            print('alpha=',model[1].coef_dist*model[1].coef_dist)
            #print('lambda=',model[1].regu*model[1].regu)
            print('target=',target)
            print('y_pred=',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('node_costs :')
            print(node_costs)
            print('nodeInsDel:',nodeInsDel.item())
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
        
        
            # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    return InsDel,nodeSub,edgeSub,loss_plt



model = nn.Sequential( OrderedDict([
        ('ComputeDist',Net(Gs,normalize=False,node_label='extended_label')),('RegressFromDist',regress(yt.to(device),20,nb_test=nb_test,device=device,weights='distance'))
#        ('RegressFromDist',RegressFromGED(yt.to(device),normalize='exp',nb_test=nb_test))
        ])) 
print('Parameters:')
for param in model.parameters():
    print(type(param), param.size())
nb_iter=40   
#with torch.autograd.profiler.profile(use_cuda=True) as prof:    
InsDel, nodeSub,edgeSub,loss_plt=regression(model,data,yt,nb_iter)
#print(prof.key_averages().table(sort_by="cpu_time_total"))
#print(prof.key_averages().table(sort_by="cuda_time_total"))
num_fig=0
plt.figure(num_fig)
plt.plot(InsDel[:,0],label="node")
plt.plot(InsDel[:,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
num_fig=num_fig+1
plt.legend()
plt.figure(num_fig)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[:,k])
plt.title('Node Substitutions costs')
num_fig=num_fig+1
if edgeSub.shape[1] != 0:
    plt.figure(num_fig)
    for k in range(edgeSub.shape[1]):
        plt.plot(edgeSub[:,k])
    plt.title('Edge Substitutions costs')
    num_fig=num_fig+1
plt.figure(num_fig)
plt.plot(loss_plt[:])
plt.title('Loss')
plt.show()
plt.close()

['C_1C', 'C_1C1C', 'C_1C1C1C', 'C_1C1C1C1C', 'C_1C1C1C1O', 'C_1C1C1C1S', 'C_1C1C1O', 'C_1C1C1O1O', 'C_1C1C1S', 'C_1C1C1S1S', 'C_1C1O', 'C_1C1O1O', 'C_1C1S', 'C_1C1S1S', 'C_1O', 'C_1O1O', 'C_1S', 'C_1S1S', 'O_1C1C', 'O_1C1O', 'S_1C1C', 'S_1C1S']
{'C_1C': 0, 'C_1C1C': 1, 'C_1C1C1C': 2, 'C_1C1C1C1C': 3, 'C_1C1C1C1O': 4, 'C_1C1C1C1S': 5, 'C_1C1C1O': 6, 'C_1C1C1O1O': 7, 'C_1C1C1S': 8, 'C_1C1C1S1S': 9, 'C_1C1O': 10, 'C_1C1O1O': 11, 'C_1C1S': 12, 'C_1C1S1S': 13, 'C_1O': 14, 'C_1O1O': 15, 'C_1S': 16, 'C_1S1S': 17, 'O_1C1C': 18, 'O_1C1O': 19, 'S_1C1C': 20, 'S_1C1S': 21} 22
nb labels: 22  ; nb_edge_labels: 1
torch.Size([164, 121])
adjacency matrices tensor([[0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 1,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 1, 0]], device='cuda:0', dtype=torch.int32)
node labels tensor([[14, 14, 19,  ...,  0,  0,  0],
        [16, 16, 20,  ...,  0,  0,  0],
      

In [ ]:
plt.plot(loss_plt[0:160])
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('Loss')
plt.show()
plt.close()
print(nodeSub[:,25])
plt.figure(0)
for k in range(nodeSub.shape[1]):    
    plt.plot(nodeSub[:,k])
plt.title('Node Substitutions costs')
plt.show()
plt.close()
print(edgeSub.shape[1])